In [1]:
import pandas as pd

In [2]:
from amplpy import AMPL,Environment
ampl = AMPL(Environment('/home/asino/Downloads/ampl_linux-intel64/'))
ampl.setOption("solver","/home/asino/Downloads/ampl_linux-intel64/cplex")
ampl.setOption("absmipgap",1)



In [3]:
ME="bryan"

In [4]:
bought=pd.read_csv("availability-of-players.csv")
dictOfMyPlayers={}
dictOfBrokenPlayers={}
for idx,player in bought.iterrows():
    if player["Owner"]==ME:
        if player["CanPlay"]==0:
            dictOfBrokenPlayers[player["Name"]]=1
        else:
            dictOfBrokenPlayers[player["Name"]]=0
print(dictOfBrokenPlayers)   

{'DESILVESTRI': 0, 'ROMAGNOLIA': 0, 'PELLEGRINILO': 0, 'RUIZ': 0, 'PINAMONTI': 0, 'MANOLAS': 0, 'KOLAROV': 0, 'ALLAN': 0, 'BARELLA': 0, 'VERDI': 0, 'CRISTIANORONALDO': 0, 'BIRAGHI': 0, 'MALINOVSKYI': 0, 'LUCASLEIVA': 0, 'BONAVENTURA': 0, 'JUWARA': 0, 'GODIN': 0, 'FAZIO': 0, 'BRANCOLINI': 1, 'PETAGNA': 0, 'TOLOI': 0, 'RAMSEY': 0, 'SIRIGU': 0, 'MERET': 0, 'RABIOT': 0, 'OSPINA': 0, 'MURIEL': 0, 'RIBERY': 0}


In [5]:
ampl.read("07-prepare-selection.mod")
ampl.readData("operations-mantra-online.dat")

	07-prepare-selection.mod
	line 33 offset 991
	Caution: 0-dimensional slice


In [6]:
def suggestTeam(dictOfBrokenPlayers):
    plas=ampl.getSet("Players").getValues()
    valuesForBrokenPlayersAmpl={}
    for name in plas:
        name=name[0]
        #print(name)
        valuesForBrokenPlayersAmpl[name]=0
    for player,value in dictOfBrokenPlayers.items():
        valuesForBrokenPlayersAmpl[player]=value
    ampl.getParameter("broken").setValues(valuesForBrokenPlayersAmpl)
        
        
        
    
    ampl.solve()


In [7]:
print(dictOfMyPlayers)
suggestTeam(dictOfBrokenPlayers)
#suggestTeam({},{})

{}
CPLEX 12.8.0.0: optimal integer solution; objective 75.28095488
1655 MIP simplex iterations
0 branch-and-bound nodes


In [8]:
putPlayerInSlot=ampl.getVariable("putPlayerInSlot")
cost=ampl.getParameter("cost")
pfs=ampl.getParameter("performance")
moduleChosen=ampl.getVariable("bidderChoosesModule")

roles=ampl.getSet("CompatibilityPlayerRole").getValues().toDict()
slots=ampl.getSet("CompatibilitySlotRole").getValues().toDict()
solution=putPlayerInSlot.getValues().toDict()
costs=cost.getValues().toDict()
pfs=pfs.getValues().toDict()
modules=moduleChosen.getValues().toDict()
moduleUsed="AAA"


for moduleName,taken in modules.items():
    if taken>0.5:
        #print("Module: {}\n\n".format(moduleName))
        moduleUsed=moduleName
print(moduleUsed)
myPls={}
for key,val in solution.items():
    
    if key[0]==moduleUsed:
        if val>0.5:
            myPls[key[2]]=key[1]
            #print(key,val)
rolesA={}

#To show the roles of each player
for key,val in roles.items():
    if key[0] not in rolesA.keys():
        rolesA[key[0]]=[]
    rolesA[key[0]].append(key[1])

    
mappingSlotNumberToPossibleRoles={}
for key,val in slots.items():
    if key[0]==moduleUsed:
        
        slotNr=key[1]
        role=key[2]
        if slotNr not in mappingSlotNumberToPossibleRoles.keys():
            mappingSlotNumberToPossibleRoles[slotNr]=[]
        mappingSlotNumberToPossibleRoles[slotNr].append(role)

emptySlotsInField=0
emptySlotsOutside=0
for slotNr,possibleRoles in mappingSlotNumberToPossibleRoles.items():
    if slotNr<=11:
        privilege="TOP"
    else:
        privilege="NO"
    try:
        print("{} plays  in slot {}  {}".format(myPls[slotNr], possibleRoles,privilege))
    except KeyError:
        if privilege=="TOP":
            emptySlotsInField+=1
        else:
            emptySlotsOutside+=1
print("Empty slots IN field: {}".format(emptySlotsInField))
print("Empty slots out field: {}".format(emptySlotsOutside))

m3511
MANOLAS plays  in slot ['Dc']  TOP
ROMAGNOLIA plays  in slot ['Dc']  TOP
FAZIO plays  in slot ['Dc']  TOP
ALLAN plays  in slot ['M']  TOP
BARELLA plays  in slot ['M']  TOP
KOLAROV plays  in slot ['E', 'W']  TOP
BIRAGHI plays  in slot ['E', 'W']  TOP
RUIZ plays  in slot ['C']  TOP
CRISTIANORONALDO plays  in slot ['A', 'Pc']  TOP
RAMSEY plays  in slot ['T', 'A']  TOP
SIRIGU plays  in slot ['Por']  TOP
GODIN plays  in slot ['Dc']  NO
TOLOI plays  in slot ['Dc']  NO
LUCASLEIVA plays  in slot ['M']  NO
DESILVESTRI plays  in slot ['E', 'W']  NO
RIBERY plays  in slot ['E', 'W']  NO
RABIOT plays  in slot ['C']  NO
PETAGNA plays  in slot ['A', 'Pc']  NO
PELLEGRINILO plays  in slot ['T', 'A']  NO
OSPINA plays  in slot ['Por']  NO
Empty slots IN field: 0
Empty slots out field: 2
